# Important libraries

In [436]:
import win32com.client
import sys
import time
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Openserver Setup

Openserver class with methods to connect to the server, send commands, and disconnect from the server.

In [437]:
class OpenServer():
    "Class for holding ActiveX reference. Allows license disconnectio"
    def __init__(self):
        self.status = "Disconnected"
        self.OSReference = None
    
    def Connect(self):
        self.OSReference = win32com.client.Dispatch("PX32.OpenServer.1")
        self.status = "Connected"
        print("OpenServer connected")
        
    def Disconnect(self):
        self.OSReference = None
        self.status = "Disconnected"
        print("OpenServer disconnected")

In [438]:
def GetAppName(sv):
    # function for returning app name from tag string
    pos = sv.find(".")
    if pos < 2:
        sys.exit("GetAppName: Badly formed tag string")
    app_name = sv[:pos]
    if app_name.lower() not in ["prosper", "mbal", "gap", "pvt", "resolve",
                                   "reveal"]:
        sys.exit("GetAppName: Unrecognised application name in tag string")
    return app_name


def DoCmd(OpenServe, cmd):
    # perform a command and check for errors
    lerr = OpenServe.OSReference.DoCommand(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoCmd: " + err)


def DoSet(OpenServe, sv, val):
    # set a value and check for errors
    lerr = OpenServe.OSReference.SetValue(sv, val)
    app_name = GetAppName(sv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSet: " + err)
    
def DoGet(OpenServe, gv):
    # get a value and check for errors
    get_value = OpenServe.OSReference.GetValue(gv)
    app_name = GetAppName(gv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGet: " + err)
    return get_value


def DoSlowCmd(OpenServe, cmd):
    # perform a command then wait for command to exit and check for errors
    step = 0.001
    app_name = GetAppName(cmd)
    lerr = OpenServe.OSReference.DoCommandAsync(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)
    while OpenServe.OSReference.IsBusy(app_name) > 0:
        if step < 2:
            step = step*2
        time.sleep(step)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)


def DoGAPFunc(OpenServe, gv):
    DoSlowCmd(gv)
    DoGAPFunc = DoGet(OpenServe, "GAP.LASTCMDRET")
    lerr = OpenServe.OSReference.GetLastError("GAP")
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGAPFunc: " + err)
    return DoGAPFunc


def OSOpenFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.OPENFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSOpenFile: " + err)


def OSSaveFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.SAVEFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSSaveFile: " + err)

# Multiple well calculation

In [439]:
#gas availability and increment steps
number_of_wells = int(input('Please enter the number of wells:'))
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
df=pd.DataFrame()
df['gas_injection_array'] = gas_injection_array
df

,gas_injection_array
0,0.0
1,0.1
2,0.2
3,0.3
4,0.4
5,0.5
6,0.6
7,0.7
8,0.8
9,0.9


In [440]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()
oil_rates_for_wells = {}

# Perform functions

cwd = os.getcwd()
for j in range(1,number_of_wells+1):
    #opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j}.Out', 'PROSPER')
    print(f'Well {j} opened')
    #oil rates calculation
    for i in range(0,len(gas_injection_array)):
        command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
        DoSet(petex, command,gas_injection_array[i])
    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_injection_array)):
        value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
        oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j}'] = oil_rates
    #closing file   
    OSSaveFile(petex, cwd + f'\well_{j}.Out', 'PROSPER')
    print(f'Well {j} closed')

#merging to main dataframe
for i in range(1,number_of_wells+1):
    df[f'Well_{i}'] = oil_rates_for_wells[f'well_{i}']

df.head()

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed


,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24


In [441]:
#sensisitivity plot if all wells receive equal amount of gas
fig = go.Figure()
for i in range(1,number_of_wells+1):
    
    fig.add_trace(go.Scatter(x=df['gas_injection_array'], y=df[f'Well_{i}'],
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot(all wells receive equal amount of gas lift)',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   )
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

In [442]:
df.head()

,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24


In [443]:
#extracting max gas lift oil rate pairs
df_new = df.drop('gas_injection_array', axis=1)
wells = []
i=1
for element in df_new.idxmax():
    oil_rate = df_new[f'Well_{i}'][element]
    gas_lift = df['gas_injection_array'][element]
    wells.append({'production_rate':oil_rate, 'gas_lift':gas_lift})
    i+=1
wells

[{'production_rate': 987.44, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1717.55, 'gas_lift': 1.4000000000000001},
 {'production_rate': 1643.5, 'gas_lift': 1.5},
 {'production_rate': 1448.2, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1104.21, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1904.17, 'gas_lift': 1.2000000000000002}]

In [444]:
# wells = [{'production_rate': 987.44, 'gas_lift': 1.7},
#          {'production_rate': 1717.55, 'gas_lift': 1.4},
#          {'production_rate': 1643.5, 'gas_lift': 1.5},
#          {'production_rate': 1448.2, 'gas_lift': 1.7},
#          {'production_rate': 1104.21, 'gas_lift': 1.9},
#          {'production_rate': 1904.17, 'gas_lift': 1.2},
#          {'production_rate': 1819.56, 'gas_lift': 1.3},
#          {'production_rate': 1225.93, 'gas_lift': 1.9},
#          {'production_rate': 1775.91, 'gas_lift': 1.3},
#          {'production_rate': 1802.58, 'gas_lift': 1.3}]
#copy and paste the values from above based on number of wells determineed
wells = [
    {'production_rate': 987.44, 'gas_lift': 1.7},
    {'production_rate': 1717.55, 'gas_lift': 1.4},
    {'production_rate': 1643.5, 'gas_lift': 1.5},
    {'production_rate': 1448.2, 'gas_lift': 1.7},
    {'production_rate': 1104.21, 'gas_lift': 1.9},
    {'production_rate': 1904.17, 'gas_lift': 1.2},
]
well_list = [(d['production_rate'], d['gas_lift']) for d in wells]

# Ant Colony Algorithm(ACO)

# Current solution with ACO

In [445]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells using an ant colony algorithm.

    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in STB/day)
        - 'gas_lift': the gas lift of the well (in STB/day)
    - gas_capacity: the total gas capacity available for allocation (in STB/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for

    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in STB/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['production_rate'] / well['gas_lift'] for well in wells])

    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))

        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))

            # Allocate gas to the well based on the allocation probability
            # allocation = min(well['gas_lift'], remaining_gas * allocation_probability)
            allocation = min(remaining_gas, remaining_gas * allocation_probability)
            allocations[j] = allocation
            remaining_gas -= allocation

        # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)

        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']

    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]


# Testing the current solution

In [446]:
gas_capacity = 5 #total available gas
#best solution parameters
alpha = 0.05
beta = 2
evaporation_rate = 0.7
iteration_count = 50

# Optimize the allocation of gas to the wells
allocations = optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count)

# Print the optimized allocation
print(allocations)


[{'well_id': 0, 'allocation': 0.24812319875248218}, {'well_id': 1, 'allocation': 1.1024697670273906}, {'well_id': 2, 'allocation': 0.6596640295600968}, {'well_id': 3, 'allocation': 0.3169318718140276}, {'well_id': 4, 'allocation': 0.12772985020437203}, {'well_id': 5, 'allocation': 0.9768376277745121}]


In [447]:
#total gas consumed
sum_gas = 0
for alloc in allocations:
    sum_gas+=alloc['allocation']
print(sum_gas)

3.4317563451328814


# Visualize the current solution

In [448]:
gas_dict = {}
for i in range(len(allocations)):
    gas_available = allocations[i]['allocation']
    increment = increment
    gas_injection_array = np.arange(0, gas_available, increment)
    gas_dict[f'gas_injection_array_{i+1}'] = gas_injection_array
gas_dict

{'gas_injection_array_1': array([0. , 0.1, 0.2]),
 'gas_injection_array_2': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1]),
 'gas_injection_array_3': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]),
 'gas_injection_array_4': array([0. , 0.1, 0.2, 0.3]),
 'gas_injection_array_5': array([0. , 0.1]),
 'gas_injection_array_6': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])}

In [449]:
# Initialises an 'OpenServer' class
petex = OpenServer()

petex.Connect()
oil_rates_for_wells = {}

# Perform functions
cwd = os.getcwd()

for j in range(len(gas_dict)):
#opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} opened')

    for i in range(0,len(gas_dict[f'gas_injection_array_{j+1}'])):
            command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
            DoSet(petex, command,gas_dict[f'gas_injection_array_{j+1}'][i])

    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_dict[f'gas_injection_array_{j+1}'])):
            value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
            oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j+1}'] = oil_rates

    OSSaveFile(petex, cwd + f'\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} closed')

oil_rates_for_wells

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed


{'well_1': [878.37, 896.05, 911.9],
 'well_2': [1657.35,
  1668.4,
  1677.71,
  1685.58,
  1692.24,
  1697.85,
  1702.57,
  1706.89,
  1710.09,
  1712.6,
  1714.54,
  1715.98],
 'well_3': [1576.44, 1588.09, 1597.9, 1606.21, 1613.28, 1619.15, 1623.94],
 'well_4': [1362.42, 1375.9, 1387.41, 1397.28],
 'well_5': [997.05, 1013.68],
 'well_6': [1858.4,
  1867.23,
  1874.69,
  1880.97,
  1886.24,
  1890.55,
  1895.85,
  1898.62,
  1900.76,
  1902.34]}

In [450]:
fig = go.Figure()
for i in range(1,number_of_wells+1):

    fig.add_trace(go.Scatter(x=gas_dict[f'gas_injection_array_{i}'], y=oil_rates_for_wells[f'well_{i}'],
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot with ACO algorithm',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5,
                   )
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

# Particle Swarm Optimization (PSO)

# Current solution with PSO

In [451]:
import numpy as np
import random

def optimize_gas_allocation_PSO(wells, gas_capacity, iteration_count, num_particles):
    # Initialize the particles and their velocities
    particles = [np.random.rand(len(wells)) for _ in range(num_particles)]
    velocities = [np.zeros(len(wells)) for _ in range(num_particles)]

    # Initialize the best position and best fitness for each particle
    personal_best_positions = particles.copy()
    personal_best_fitness = [np.inf for _ in range(num_particles)]

    # Initialize the global best position and global best fitness
    global_best_position = personal_best_positions[np.argmin(personal_best_fitness)]
    global_best_fitness = np.inf

    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Update the position and fitness of each particle
        for j, particle in enumerate(particles):
            # Update the velocity of the particle
            rp = random.uniform(0,1)
            rg = random.uniform(0,1)
            velocities[j] = 0.5 * velocities[j] + rp * (personal_best_positions[j] - particle) + rg * (global_best_position - particle)
            # Update the position of the particle
            particles[j] += velocities[j]
            # Ensure that the particle's position is within the constraints
            particles[j] = np.maximum(np.zeros(len(wells)), particles[j])

            # Calculate the fitness of the particle
            fitness = -1 * np.sum(particles[j] * [well['production_rate'] for well in wells])
            # Update the personal best position and personal best fitness of the particle
            if fitness < personal_best_fitness[j]:
                personal_best_positions[j] = particles[j]
                personal_best_fitness[j] = fitness

            # Check if the total gas allocated to all wells is less than or equal to the total available gas capacity
            total_gas_allocated = sum(particles[j])
            if total_gas_allocated > gas_capacity:
                particles[j] = particles[j] * (gas_capacity / total_gas_allocated)
                total_gas_allocated = gas_capacity

            # Update the global best position and global best fitness
            if fitness < global_best_fitness:
                global_best_position = particles[j]
                global_best_fitness = fitness
    return [{'well_id': j, 'allocation': global_best_position[j]} for j in range(len(wells))]


# Testing the current solution

In [452]:
gas_capacity= 5
iteration_count= 50
num_part = 240
allocations = optimize_gas_allocation_PSO(wells, gas_capacity, iteration_count, num_part)
allocations

[{'well_id': 0, 'allocation': 0.8172440007001558},
 {'well_id': 1, 'allocation': 1.0535139276545769},
 {'well_id': 2, 'allocation': 0.5033719289622917},
 {'well_id': 3, 'allocation': 1.2691104309174006},
 {'well_id': 4, 'allocation': 0.3188801257995092},
 {'well_id': 5, 'allocation': 1.0378795859660652}]

In [453]:
#total gas consumed
sum_gas = 0
for alloc in allocations:
    sum_gas+=alloc['allocation']
print(sum_gas)

5.0


# Visualize the current solution

In [454]:
gas_dict = {}
for i in range(len(allocations)):
    gas_available = allocations[i]['allocation']
    increment = increment
    gas_injection_array = np.arange(0, gas_available, increment)
    gas_dict[f'gas_injection_array_{i+1}'] = gas_injection_array
gas_dict

{'gas_injection_array_1': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
 'gas_injection_array_2': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'gas_injection_array_3': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5]),
 'gas_injection_array_4': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2]),
 'gas_injection_array_5': array([0. , 0.1, 0.2, 0.3]),
 'gas_injection_array_6': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])}

In [455]:
# Initialises an 'OpenServer' class
petex = OpenServer()

petex.Connect()
oil_rates_for_wells = {}

# Perform functions
cwd = os.getcwd()

for j in range(len(gas_dict)):
#opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} opened')

    for i in range(0,len(gas_dict[f'gas_injection_array_{j+1}'])):
            command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
            DoSet(petex, command,gas_dict[f'gas_injection_array_{j+1}'][i])

    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_dict[f'gas_injection_array_{j+1}'])):
            value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
            oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j+1}'] = oil_rates

    OSSaveFile(petex, cwd + f'\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} closed')

oil_rates_for_wells

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed


{'well_1': [878.37,
  896.05,
  911.9,
  924.19,
  935.19,
  944.14,
  951.77,
  958.21,
  963.73],
 'well_2': [1657.35,
  1668.4,
  1677.71,
  1685.58,
  1692.24,
  1697.85,
  1702.57,
  1706.89,
  1710.09,
  1712.6,
  1714.54],
 'well_3': [1576.44, 1588.09, 1597.9, 1606.21, 1613.28, 1619.15],
 'well_4': [1362.42,
  1375.9,
  1387.41,
  1397.28,
  1405.79,
  1413.16,
  1419.53,
  1425.0,
  1431.86,
  1435.76,
  1438.99,
  1441.7,
  1443.93],
 'well_5': [997.05, 1013.68, 1027.6, 1041.02],
 'well_6': [1858.4,
  1867.23,
  1874.69,
  1880.97,
  1886.24,
  1890.55,
  1895.85,
  1898.62,
  1900.76,
  1902.34,
  1903.41]}

In [456]:
fig = go.Figure()
for i in range(1,number_of_wells+1):

    fig.add_trace(go.Scatter(x=gas_dict[f'gas_injection_array_{i}'], y=oil_rates_for_wells[f'well_{i}'],
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot with PSO algorithm',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5,
                   )
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()